In [1]:
#import 
import numpy as np
import pandas as pd
import hyperopt
from catboost import Pool, CatBoostClassifier, cv
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
import lightgbm
from lightgbm import LGBMClassifier

In [2]:
import sys
!{sys.executable} -m pip install hyperopt

In [2]:
train = pd.read_csv("C:/Users/USER.USER-PC.002/Desktop/Capstone Data/train_values.csv")
test = pd.read_csv("C:/Users/USER.USER-PC.002/Desktop/Capstone Data/test_values.csv")
train_label = pd.read_csv("C:/Users/USER.USER-PC.002/Desktop/Capstone Data/train_labels.csv")

In [3]:
join_train = pd.merge(train, train_label, on='row_id')
join_train.head()

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,0,3,1,1,1,70.0,3,18,37,246,...,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0,4536,False,1
1,1,1,1,3,1,178.0,3,369,52,299,...,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0,2458,False,0
2,2,2,1,3,1,163.0,3,16,10,306,...,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0,5710,False,1
3,3,1,1,1,1,155.0,1,305,47,180,...,105.0,6667.0,6.246,78439.0,100.000,2025.0,2299.0,5888,True,1
4,4,1,1,1,1,305.0,3,24,37,20,...,71.0,6732.0,100.000,63075.0,82.200,1464.0,1847.0,289,False,1


In [4]:
join_train.isnull().sum()

row_id                                0
loan_type                             0
property_type                         0
loan_purpose                          0
occupancy                             0
loan_amount                           0
preapproval                           0
msa_md                                0
state_code                            0
county_code                           0
applicant_ethnicity                   0
applicant_race                        0
applicant_sex                         0
applicant_income                  39948
population                        22465
minority_population_pct           22466
ffiecmedian_family_income         22440
tract_to_msa_md_income_pct        22514
number_of_owner-occupied_units    22565
number_of_1_to_4_family_units     22530
lender                                0
co_applicant                          0
accepted                              0
dtype: int64

In [5]:
join_train.fillna(-999,inplace=True)
test.fillna(-999,inplace=True)

In [6]:
mean_encoding = join_train.groupby(['lender']).agg({'accepted':['mean']}).reset_index()

In [9]:
mean_encoding

,lender,accepted
,,mean
0,0,1.000000
1,1,1.000000
2,2,1.000000
3,4,0.500000
4,5,0.333333
5,6,0.333333
6,7,1.000000
7,8,1.000000
8,9,0.470588


In [7]:
join_train = join_train.merge(mean_encoding, on='lender',how='left')

C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\pandas\core\reshape\merge.py:522: UserWarning: merging between different levels can give an unintended result (1 levels on the left, 2 on the right)
  warnings.warn(msg, UserWarning)
C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\pandas\core\generic.py:3812: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  new_axis = axis.drop(labels, errors=errors)


In [8]:
test = test.merge(mean_encoding, on='lender',how='left')

In [7]:
join_train

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant,accepted
0,0,3,1,1,1,70.0,3,18,37,246,...,24.0,6203.0,44.230,60588.0,50.933,716.0,2642.0,4536,False,1
1,1,1,1,3,1,178.0,3,369,52,299,...,57.0,5774.0,15.905,54821.0,100.000,1622.0,2108.0,2458,False,0
2,2,2,1,3,1,163.0,3,16,10,306,...,67.0,6094.0,61.270,67719.0,100.000,760.0,1048.0,5710,False,1
3,3,1,1,1,1,155.0,1,305,47,180,...,105.0,6667.0,6.246,78439.0,100.000,2025.0,2299.0,5888,True,1
4,4,1,1,1,1,305.0,3,24,37,20,...,71.0,6732.0,100.000,63075.0,82.200,1464.0,1847.0,289,False,1
5,5,1,1,3,1,133.0,3,221,13,55,...,51.0,6078.0,4.821,82745.0,96.550,1827.0,2340.0,964,False,1
6,6,3,1,1,1,240.0,3,374,28,131,...,104.0,6068.0,26.427,65282.0,81.068,1863.0,2560.0,5488,False,1
7,7,2,1,1,1,210.0,3,322,37,35,...,55.0,6030.0,78.153,108353.0,59.439,969.0,1601.0,2442,True,1
8,8,1,1,3,2,209.0,3,24,37,20,...,244.0,5151.0,88.156,63414.0,62.813,411.0,481.0,2118,True,1
9,9,1,1,3,1,197.0,3,194,9,20,...,86.0,7916.0,24.893,65927.0,100.000,1861.0,2123.0,3507,False,0


In [8]:
test

,row_id,loan_type,property_type,loan_purpose,occupancy,loan_amount,preapproval,msa_md,state_code,county_code,...,applicant_sex,applicant_income,population,minority_population_pct,ffiecmedian_family_income,tract_to_msa_md_income_pct,number_of_owner-occupied_units,number_of_1_to_4_family_units,lender,co_applicant
0,0,2,1,3,1,115.0,3,101,16,276,...,1,-999.0,6329.0,59.536,69889.0,85.780,1874.0,2410.0,3791,True
1,1,1,1,1,1,252.0,2,87,20,68,...,1,107.0,2473.0,8.050,65313.0,100.000,947.0,1214.0,2839,True
2,2,1,1,1,1,270.0,1,-1,-1,-1,...,2,119.0,-999.0,-999.000,-999.0,-999.000,-999.0,-999.0,4701,False
3,3,2,1,1,1,179.0,2,376,20,11,...,2,44.0,4795.0,29.676,57766.0,100.000,1426.0,1765.0,2153,True
4,4,2,1,1,1,36.0,2,254,48,156,...,3,32.0,5246.0,5.110,63332.0,100.000,1452.0,2092.0,5710,False
5,5,1,1,3,1,65.0,3,8,15,131,...,2,60.0,4502.0,9.626,61214.0,100.000,1487.0,1764.0,5513,True
6,6,1,1,1,1,402.0,2,-1,28,-1,...,1,120.0,-999.0,-999.000,-999.0,-999.000,-999.0,-999.0,3560,True
7,7,1,1,3,1,200.0,3,275,48,123,...,3,35.0,5256.0,23.770,97181.0,100.000,1685.0,1760.0,763,False
8,8,1,1,1,2,121.0,3,350,38,233,...,1,309.0,3231.0,21.712,85665.0,83.509,966.0,1282.0,5498,False
9,9,3,1,1,1,213.0,3,186,34,20,...,1,72.0,8866.0,39.604,101894.0,97.009,2221.0,2896.0,1574,False


In [6]:
x = join_train.drop('accepted',axis=1)
y = join_train.accepted

In [10]:
x = x.drop('lender',axis=1)
test = test.drop('lender',axis=1)

In [52]:
test.columns

Index(['row_id', 'loan_type', 'property_type', 'loan_purpose', 'occupancy',
       'loan_amount', 'preapproval', 'msa_md', 'state_code', 'county_code',
       'applicant_ethnicity', 'applicant_race', 'applicant_sex',
       'applicant_income', 'population', 'minority_population_pct',
       'ffiecmedian_family_income', 'tract_to_msa_md_income_pct',
       'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
       'lender', 'co_applicant'],
      dtype='object')

In [53]:
x.columns

Index(['row_id', 'loan_type', 'property_type', 'loan_purpose', 'occupancy',
       'loan_amount', 'preapproval', 'msa_md', 'state_code', 'county_code',
       'applicant_ethnicity', 'applicant_race', 'applicant_sex',
       'applicant_income', 'population', 'minority_population_pct',
       'ffiecmedian_family_income', 'tract_to_msa_md_income_pct',
       'number_of_owner-occupied_units', 'number_of_1_to_4_family_units',
       'lender', 'co_applicant'],
      dtype='object')

In [54]:
x.dtypes


row_id                              int64
loan_type                           int64
property_type                       int64
loan_purpose                        int64
occupancy                           int64
loan_amount                       float64
preapproval                         int64
msa_md                              int64
state_code                          int64
county_code                         int64
applicant_ethnicity                 int64
applicant_race                      int64
applicant_sex                       int64
applicant_income                  float64
population                        float64
minority_population_pct           float64
ffiecmedian_family_income         float64
tract_to_msa_md_income_pct        float64
number_of_owner-occupied_units    float64
number_of_1_to_4_family_units     float64
lender                              int64
co_applicant                         bool
dtype: object

In [7]:
cate_features_index = np.where(x.dtypes != float)[0]

In [8]:
xtrain,xtest,ytrain,ytest = train_test_split(x,y,test_size=0.15)

In [10]:
train_data = lightgbm.Dataset(xtrain, label=ytrain, categorical_feature=cate_features_index)
test_data = lightgbm.Dataset(xtest, label=ytest)

In [11]:
categorical_features = [c for c, col in enumerate(join_train.columns) if 'cat' in col]

In [18]:
pd.DataFrame(train_data)

ValueError: DataFrame constructor not properly called!

In [12]:
parameters = {
    'application': 'binary',
    'objective': 'binary',
    'metric': 'Accuracy',
    'is_unbalance': 'true',
    'boosting': 'gbdt',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': 0.01,
    'verbose': 0
}

In [13]:
model = lightgbm.train(parameters,
                       train_data,
                       valid_sets=test_data,
                       num_boost_round=1000,
                       early_stopping_rounds=100)

C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\lightgbm\basic.py:1198: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self.categorical_feature == categorical_feature:
C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\lightgbm\basic.py:1205: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\lightgbm\basic.py:1204: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif categorical_feature == 'auto':
C:\Users\USER.USER-PC.002\Anaconda3\lib\site-packages\lightgbm\basic.py:1209: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 12, 20, 21]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\USER.USER-PC.002

TypeError: Wrong type(int64) or unknown name(0) in categorical_feature

In [9]:
model = CatBoostClassifier(iterations=15000, depth=8, learning_rate=0.01,eval_metric='Accuracy',use_best_model=True, random_seed=42)

In [ ]:
model.fit(xtrain,ytrain,cat_features=cate_features_index,eval_set=(xtest,ytest))


0:	learn: 0.7012447	test: 0.7010000	best: 0.7010000 (0)	total: 3.37s	remaining: 14h 1m 52s
1:	learn: 0.7022400	test: 0.7026800	best: 0.7026800 (1)	total: 5.96s	remaining: 12h 25m 27s
2:	learn: 0.7030000	test: 0.7022933	best: 0.7026800 (1)	total: 8.77s	remaining: 12h 10m 27s
3:	learn: 0.7028447	test: 0.7015733	best: 0.7026800 (1)	total: 11.9s	remaining: 12h 21m 20s
4:	learn: 0.7027318	test: 0.7027333	best: 0.7027333 (4)	total: 14.5s	remaining: 12h 6m 24s
5:	learn: 0.7030965	test: 0.7032267	best: 0.7032267 (5)	total: 17s	remaining: 11h 46m 18s
6:	learn: 0.7039412	test: 0.7042400	best: 0.7042400 (6)	total: 20.1s	remaining: 11h 57m 39s
7:	learn: 0.7042282	test: 0.7044133	best: 0.7044133 (7)	total: 23.3s	remaining: 12h 6m 37s
8:	learn: 0.7050941	test: 0.7052533	best: 0.7052533 (8)	total: 26.1s	remaining: 12h 5m 41s
9:	learn: 0.7052706	test: 0.7054000	best: 0.7054000 (9)	total: 28.9s	remaining: 12h 1m 23s
10:	learn: 0.7054212	test: 0.7057600	best: 0.7057600 (10)	total: 31.3s	remaining: 11h 5

88:	learn: 0.7086071	test: 0.7092000	best: 0.7092667 (86)	total: 4m 9s	remaining: 11h 35m 47s
89:	learn: 0.7086306	test: 0.7092800	best: 0.7092800 (89)	total: 4m 11s	remaining: 11h 35m 29s
90:	learn: 0.7086588	test: 0.7093600	best: 0.7093600 (90)	total: 4m 14s	remaining: 11h 34m 53s
91:	learn: 0.7088729	test: 0.7094667	best: 0.7094667 (91)	total: 4m 17s	remaining: 11h 36m 5s
92:	learn: 0.7089694	test: 0.7093867	best: 0.7094667 (91)	total: 4m 20s	remaining: 11h 37m 7s
93:	learn: 0.7091976	test: 0.7096400	best: 0.7096400 (93)	total: 4m 24s	remaining: 11h 37m 52s
94:	learn: 0.7093529	test: 0.7097200	best: 0.7097200 (94)	total: 4m 26s	remaining: 11h 37m 33s
95:	learn: 0.7094188	test: 0.7099733	best: 0.7099733 (95)	total: 4m 29s	remaining: 11h 37m 57s
96:	learn: 0.7095647	test: 0.7101733	best: 0.7101733 (96)	total: 4m 32s	remaining: 11h 37m 12s
97:	learn: 0.7097576	test: 0.7104000	best: 0.7104000 (97)	total: 4m 34s	remaining: 11h 36m 14s
98:	learn: 0.7098094	test: 0.7104267	best: 0.7104267 

173:	learn: 0.7137082	test: 0.7143067	best: 0.7144133 (172)	total: 8m 22s	remaining: 11h 52m 53s
174:	learn: 0.7136659	test: 0.7143867	best: 0.7144133 (172)	total: 8m 24s	remaining: 11h 52m 54s
175:	learn: 0.7137129	test: 0.7145333	best: 0.7145333 (175)	total: 8m 28s	remaining: 11h 53m 8s
176:	learn: 0.7139012	test: 0.7145067	best: 0.7145333 (175)	total: 8m 31s	remaining: 11h 53m 26s
177:	learn: 0.7139247	test: 0.7145067	best: 0.7145333 (175)	total: 8m 34s	remaining: 11h 53m 42s
178:	learn: 0.7139741	test: 0.7145333	best: 0.7145333 (175)	total: 8m 37s	remaining: 11h 54m 19s
179:	learn: 0.7140329	test: 0.7146133	best: 0.7146133 (179)	total: 8m 40s	remaining: 11h 54m 31s
180:	learn: 0.7140588	test: 0.7147333	best: 0.7147333 (180)	total: 8m 43s	remaining: 11h 54m 42s
181:	learn: 0.7141271	test: 0.7149467	best: 0.7149467 (181)	total: 8m 47s	remaining: 11h 55m 25s
182:	learn: 0.7141294	test: 0.7148667	best: 0.7149467 (181)	total: 8m 50s	remaining: 11h 55m 24s
183:	learn: 0.7141341	test: 0.7

258:	learn: 0.7167929	test: 0.7166800	best: 0.7168000 (256)	total: 13m 1s	remaining: 12h 21m 16s
259:	learn: 0.7168471	test: 0.7166800	best: 0.7168000 (256)	total: 13m 4s	remaining: 12h 21m 5s
260:	learn: 0.7168329	test: 0.7167733	best: 0.7168000 (256)	total: 13m 6s	remaining: 12h 20m 41s
261:	learn: 0.7168612	test: 0.7167333	best: 0.7168000 (256)	total: 13m 10s	remaining: 12h 21m 3s
262:	learn: 0.7168776	test: 0.7167067	best: 0.7168000 (256)	total: 13m 13s	remaining: 12h 21m 3s
263:	learn: 0.7169624	test: 0.7166667	best: 0.7168000 (256)	total: 13m 16s	remaining: 12h 20m 54s
264:	learn: 0.7169741	test: 0.7168533	best: 0.7168533 (264)	total: 13m 19s	remaining: 12h 20m 48s
265:	learn: 0.7169906	test: 0.7167467	best: 0.7168533 (264)	total: 13m 22s	remaining: 12h 20m 30s
266:	learn: 0.7170353	test: 0.7166667	best: 0.7168533 (264)	total: 13m 25s	remaining: 12h 20m 39s
267:	learn: 0.7170706	test: 0.7166000	best: 0.7168533 (264)	total: 13m 28s	remaining: 12h 20m 23s
268:	learn: 0.7170588	test

342:	learn: 0.7193012	test: 0.7183333	best: 0.7183333 (342)	total: 17m 20s	remaining: 12h 21m 6s
343:	learn: 0.7192988	test: 0.7182933	best: 0.7183333 (342)	total: 17m 23s	remaining: 12h 20m 38s
344:	learn: 0.7192988	test: 0.7182400	best: 0.7183333 (342)	total: 17m 26s	remaining: 12h 20m 34s
345:	learn: 0.7193365	test: 0.7183467	best: 0.7183467 (345)	total: 17m 28s	remaining: 12h 20m 16s
346:	learn: 0.7193035	test: 0.7183067	best: 0.7183467 (345)	total: 17m 31s	remaining: 12h 20m 23s
347:	learn: 0.7192847	test: 0.7183733	best: 0.7183733 (347)	total: 17m 34s	remaining: 12h 20m 12s
348:	learn: 0.7193224	test: 0.7183733	best: 0.7183733 (347)	total: 17m 37s	remaining: 12h 20m 2s
349:	learn: 0.7193882	test: 0.7184267	best: 0.7184267 (349)	total: 17m 40s	remaining: 12h 19m 59s
350:	learn: 0.7193906	test: 0.7184267	best: 0.7184267 (349)	total: 17m 43s	remaining: 12h 20m 5s
351:	learn: 0.7194494	test: 0.7185600	best: 0.7185600 (351)	total: 17m 47s	remaining: 12h 20m 2s
352:	learn: 0.7194941	te

...

1993:	learn: 0.7342282	test: 0.7290533	best: 0.7291467 (1981)	total: 1h 40m 25s	remaining: 10h 55m 3s
1994:	learn: 0.7342659	test: 0.7290533	best: 0.7291467 (1981)	total: 1h 40m 28s	remaining: 10h 55m
1995:	learn: 0.7342682	test: 0.7290533	best: 0.7291467 (1981)	total: 1h 40m 32s	remaining: 10h 54m 59s
1996:	learn: 0.7342682	test: 0.7291067	best: 0.7291467 (1981)	total: 1h 40m 35s	remaining: 10h 54m 59s
1997:	learn: 0.7343012	test: 0.7291467	best: 0.7291467 (1981)	total: 1h 40m 38s	remaining: 10h 54m 56s
1998:	learn: 0.7342753	test: 0.7291600	best: 0.7291600 (1998)	total: 1h 40m 42s	remaining: 10h 55m
1999:	learn: 0.7342918	test: 0.7291600	best: 0.7291600 (1998)	total: 1h 40m 45s	remaining: 10h 54m 58s
2000:	learn: 0.7342965	test: 0.7291733	best: 0.7291733 (2000)	total: 1h 40m 48s	remaining: 10h 54m 55s
2001:	learn: 0.7343106	test: 0.7291867	best: 0.7291867 (2001)	total: 1h 40m 52s	remaining: 10h 54m 53s
2002:	learn: 0.7343082	test: 0.7291200	best: 0.7291867 (2001)	total: 1h 40m 55s	re

2073:	learn: 0.7346941	test: 0.7293600	best: 0.7293600 (2073)	total: 1h 44m 30s	remaining: 10h 51m 18s
2074:	learn: 0.7346918	test: 0.7293200	best: 0.7293600 (2073)	total: 1h 44m 33s	remaining: 10h 51m 16s
2075:	learn: 0.7347153	test: 0.7293333	best: 0.7293600 (2073)	total: 1h 44m 36s	remaining: 10h 51m 13s
2076:	learn: 0.7347365	test: 0.7292933	best: 0.7293600 (2073)	total: 1h 44m 39s	remaining: 10h 51m 8s
2077:	learn: 0.7347247	test: 0.7293067	best: 0.7293600 (2073)	total: 1h 44m 42s	remaining: 10h 51m 6s
2078:	learn: 0.7347553	test: 0.7294133	best: 0.7294133 (2078)	total: 1h 44m 45s	remaining: 10h 51m 1s
2079:	learn: 0.7347624	test: 0.7293333	best: 0.7294133 (2078)	total: 1h 44m 48s	remaining: 10h 50m 58s
2080:	learn: 0.7347600	test: 0.7293467	best: 0.7294133 (2078)	total: 1h 44m 51s	remaining: 10h 50m 56s
2081:	learn: 0.7347482	test: 0.7293200	best: 0.7294133 (2078)	total: 1h 44m 54s	remaining: 10h 50m 54s
2082:	learn: 0.7347529	test: 0.7293867	best: 0.7294133 (2078)	total: 1h 44m 

2153:	learn: 0.7350800	test: 0.7293867	best: 0.7295467 (2126)	total: 1h 48m 29s	remaining: 10h 46m 58s
2154:	learn: 0.7350682	test: 0.7293867	best: 0.7295467 (2126)	total: 1h 48m 31s	remaining: 10h 46m 54s
2155:	learn: 0.7350518	test: 0.7294267	best: 0.7295467 (2126)	total: 1h 48m 35s	remaining: 10h 46m 53s
2156:	learn: 0.7350376	test: 0.7294267	best: 0.7295467 (2126)	total: 1h 48m 38s	remaining: 10h 46m 50s
2157:	learn: 0.7350565	test: 0.7294267	best: 0.7295467 (2126)	total: 1h 48m 41s	remaining: 10h 46m 47s
2158:	learn: 0.7350353	test: 0.7294267	best: 0.7295467 (2126)	total: 1h 48m 44s	remaining: 10h 46m 43s
2159:	learn: 0.7350635	test: 0.7294400	best: 0.7295467 (2126)	total: 1h 48m 46s	remaining: 10h 46m 38s
2160:	learn: 0.7350635	test: 0.7294133	best: 0.7295467 (2126)	total: 1h 48m 50s	remaining: 10h 46m 37s
2161:	learn: 0.7350706	test: 0.7294533	best: 0.7295467 (2126)	total: 1h 48m 53s	remaining: 10h 46m 35s
2162:	learn: 0.7350706	test: 0.7294400	best: 0.7295467 (2126)	total: 1h 4

2233:	learn: 0.7354000	test: 0.7295333	best: 0.7296267 (2226)	total: 1h 52m 31s	remaining: 10h 42m 58s
2234:	learn: 0.7354094	test: 0.7295467	best: 0.7296267 (2226)	total: 1h 52m 34s	remaining: 10h 42m 58s
2235:	learn: 0.7354259	test: 0.7295467	best: 0.7296267 (2226)	total: 1h 52m 37s	remaining: 10h 42m 53s
2236:	learn: 0.7354329	test: 0.7295333	best: 0.7296267 (2226)	total: 1h 52m 40s	remaining: 10h 42m 50s
2237:	learn: 0.7354141	test: 0.7295733	best: 0.7296267 (2226)	total: 1h 52m 43s	remaining: 10h 42m 46s
2238:	learn: 0.7354329	test: 0.7295467	best: 0.7296267 (2226)	total: 1h 52m 45s	remaining: 10h 42m 42s
2239:	learn: 0.7354329	test: 0.7294933	best: 0.7296267 (2226)	total: 1h 52m 49s	remaining: 10h 42m 40s
2240:	learn: 0.7354306	test: 0.7294533	best: 0.7296267 (2226)	total: 1h 52m 52s	remaining: 10h 42m 40s
2241:	learn: 0.7354424	test: 0.7295067	best: 0.7296267 (2226)	total: 1h 52m 55s	remaining: 10h 42m 36s
2242:	learn: 0.7354376	test: 0.7294533	best: 0.7296267 (2226)	total: 1h 5

2313:	learn: 0.7357859	test: 0.7296133	best: 0.7297333 (2276)	total: 1h 56m 29s	remaining: 10h 38m 39s
2314:	learn: 0.7357859	test: 0.7295867	best: 0.7297333 (2276)	total: 1h 56m 32s	remaining: 10h 38m 37s
2315:	learn: 0.7357976	test: 0.7296000	best: 0.7297333 (2276)	total: 1h 56m 36s	remaining: 10h 38m 35s
2316:	learn: 0.7358047	test: 0.7296667	best: 0.7297333 (2276)	total: 1h 56m 39s	remaining: 10h 38m 32s
2317:	learn: 0.7358165	test: 0.7296667	best: 0.7297333 (2276)	total: 1h 56m 41s	remaining: 10h 38m 28s
2318:	learn: 0.7358424	test: 0.7296533	best: 0.7297333 (2276)	total: 1h 56m 45s	remaining: 10h 38m 26s
2319:	learn: 0.7358471	test: 0.7296667	best: 0.7297333 (2276)	total: 1h 56m 47s	remaining: 10h 38m 22s
2320:	learn: 0.7358376	test: 0.7296933	best: 0.7297333 (2276)	total: 1h 56m 51s	remaining: 10h 38m 20s
2321:	learn: 0.7358518	test: 0.7297600	best: 0.7297600 (2321)	total: 1h 56m 53s	remaining: 10h 38m 15s
2322:	learn: 0.7358471	test: 0.7297733	best: 0.7297733 (2322)	total: 1h 5

#### mean_encoding = join_train.groupby(['lender']).agg({'accepted':['mean']}).reset_index()

In [ ]:
print('the test accuracy is :{:.6f}'.format(accuracy_score(ytest, model.predict(xtest))))

In [ ]:
pred = model.predict(test)
pred = pred.astype(np.int)
submission = pd.DataFrame({'row_id':test['row_id'],'accepted':pred})

In [ ]:
submission.to_csv('C:/Users/USER.USER-PC.002/Desktop/Capstone Data/catboost_subfirst.csv',index=False)